In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))

In [35]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [36]:
#Build the dataset

def build_dataset(words):
    block_size = 3 # context length / Hyper parameter to tune
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    n_tr = 0.8 * len(words)
    n_te = 0.9 * len(words)
    X_tr = torch.tensor(X[:n_tr])
    Y_tr = torch.tensor(Y[:n_tr])
    
    X_dev = torch.tensor(X[n_tr: n_te])
    Y_dev = torch.tensor(Y[n_tr: n_te])
    
    X_test = torch.tensor(X[n_te:])
    Y_test = torch.tensor(Y[n_te:])
    return {"training": [X_tr, Y_tr], "dev": [X_dev, Y_dev], "test": [X_test, Y_test]}


In [37]:
dataset = build_dataset(words)

TypeError: slice indices must be integers or None or have an __index__ method

In [29]:
# Training initialization
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator = g)
W1 = torch.randn((2 * len(context), 100), generator = g) 
b1 = torch.randn(100, generator = g)
W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn(27, generator = g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [30]:
sum(p.nelement() for p in parameters)

3481

In [31]:
# Training
for _ in range(1000):
    # Forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += - 0.1 * p.grad
print(loss.item())



2.5609700679779053
